In [1]:
!kaggle datasets download lotusacharya/nepalinewsdataset

Dataset URL: https://www.kaggle.com/datasets/lotusacharya/nepalinewsdataset
License(s): GPL-2.0
100%|█████████████████████████████████████▊| 18.0M/18.1M [00:01<00:00, 14.0MB/s]
100%|██████████████████████████████████████| 18.1M/18.1M [00:01<00:00, 13.2MB/s]


In [ ]:
import os
import hashlib
from tqdm import tqdm
import time

def get_file_hash(filepath):
    """Calculate MD5 hash of a file to detect duplicates"""
    hash_md5 = hashlib.md5()
    try:
        with open(filepath, "rb") as f:
            for chunk in iter(lambda: f.read(4096), b""):
                hash_md5.update(chunk)
        return hash_md5.hexdigest()
    except Exception as e:
        print(f"Error reading {filepath}: {e}")
        return None

def crawl_and_combine_txt_files(root_folder, output_file="news.txt"):
    """
    Crawl through all subfolders, find .txt files, remove duplicates, and combine them
    
    Args:
        root_folder: Path to the main folder (nepalinewsdataset)
        output_file: Name of the output combined file
    """
    
    # Dictionary to store file hashes and content (to detect duplicates)
    seen_files = {}
    txt_files = []
    total_size = 0
    
    print(f"Scanning folder structure from: {root_folder}")
    print("-" * 50)
    
    # First pass: Collect all .txt files with progress
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith('.txt'):
                filepath = os.path.join(root, file)
                txt_files.append(filepath)

    
    print(f"Found {len(txt_files)} .txt files")
    
    if not txt_files:
        print("No .txt files found!")
        return
    
    # Process files with progress bar
    unique_files = 0
    duplicate_files = 0
    combined_content = []
    
    print("\nProcessing files and removing duplicates...")
    
    with tqdm(total=len(txt_files), desc="Processing", unit="file") as pbar:
        for filepath in txt_files:
            try:
                # Get file hash to check for duplicates
                file_hash = get_file_hash(filepath)
                
                if file_hash and file_hash not in seen_files:
                    # Read file content
                    with open(filepath, 'r', encoding='utf-8') as f:
                        content = f.read().strip()
                    
                    # Only add if file has content
                    if content:
                        # Add metadata about source file
                        relative_path = os.path.relpath(filepath, root_folder)
                        combined_content.append(f"\n\n{'='*80}\n")
                        combined_content.append(f"Source: {relative_path}\n")
                        combined_content.append(f"{'='*80}\n\n")
                        combined_content.append(content)
                        
                        seen_files[file_hash] = True
                        unique_files += 1
                        total_size += len(content)
                    else:
                        print(f"\nSkipping empty file: {filepath}")
                elif file_hash:
                    duplicate_files += 1
                    
            except UnicodeDecodeError:
                # Try with different encoding if utf-8 fails
                try:
                    with open(filepath, 'r', encoding='latin-1') as f:
                        content = f.read().strip()
                    
                    if content:
                        relative_path = os.path.relpath(filepath, root_folder)
                        combined_content.append(f"\n\n{'='*80}\n")
                        combined_content.append(f"Source: {relative_path}\n")
                        combined_content.append(f"{'='*80}\n\n")
                        combined_content.append(content)
                        
                        seen_files[file_hash] = True
                        unique_files += 1
                        total_size += len(content)
                except Exception as e:
                    print(f"\nCould not read {filepath} with any encoding: {e}")
                    
            except Exception as e:
                print(f"\nError processing {filepath}: {e}")
            
            pbar.update(1)
            # Small sleep to prevent CPU overuse
            time.sleep(0.001)
    
    print(f"\n{'='*60}")
    print(f"Summary:")
    print(f"  Total files found: {len(txt_files)}")
    print(f"  Unique files: {unique_files}")
    print(f"  Duplicate files skipped: {duplicate_files}")
    print(f"  Estimated output size: {total_size / (1024*1024):.2f} MB")
    print(f"{'='*60}\n")
    
    # Write combined content to output file
    if combined_content:
        print(f"Writing combined content to: {output_file}")
        
        with open(output_file, 'w', encoding='utf-8') as outfile:
            # Write header
            outfile.write(f"{'='*80}\n")
            outfile.write(f"COMBINED NEPALI NEWS DATASET\n")
            outfile.write(f"Generated from: {root_folder}\n")
            outfile.write(f"Total unique articles: {unique_files}\n")
            outfile.write(f"Generated on: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
            outfile.write(f"{'='*80}\n")
            
            # Write all content
            for content_chunk in tqdm(combined_content, desc="Writing to file", unit="chunk"):
                outfile.write(content_chunk)
        
        print(f"\n✅ Successfully created {output_file}")
        print(f"📁 Output location: {os.path.abspath(output_file)}")
        
        # Get final file size
        output_size = os.path.getsize(output_file) / (1024*1024)
        print(f"📊 Final file size: {output_size:.2f} MB")
    else:
        print("❌ No content to write!")

def main():
    # Set your main folder path
    root_folder = "nepalinewsdataset"  # Change this if needed
    
    # Check if folder exists
    if not os.path.exists(root_folder):
        print(f"Error: Folder '{root_folder}' not found!")
        print("Please make sure you're running this script from the correct directory.")
        return
    
    # Set output file name
    output_file = "news.txt"
    
    # Check if output file already exists
    if os.path.exists(output_file):
        response = input(f"{output_file} already exists. Overwrite? (y/n): ")
        if response.lower() != 'y':
            print("Operation cancelled.")
            return
    
    # Run the crawler and combiner
    try:
        crawl_and_combine_txt_files(root_folder, output_file)
    except KeyboardInterrupt:
        print("\n\n⚠️  Process interrupted by user.")
    except Exception as e:
        print(f"\n❌ Error occurred: {e}")

if __name__ == "__main__":
    # Install tqdm if not installed
    try:
        from tqdm import tqdm
    except ImportError:
        print("Installing required package: tqdm")
        import subprocess
        subprocess.check_call(["pip", "install", "tqdm"])
        from tqdm import tqdm
    
    main()

Scanning folder structure from: nepalinewsdataset
--------------------------------------------------
Found 10000 .txt files

Processing files and removing duplicates...


Processing: 100%|██████████| 10000/10000 [00:11<00:00, 847.54file/s]



Summary:
  Total files found: 10000
  Unique files: 9999
  Duplicate files skipped: 1
  Estimated output size: 26.52 MB

Writing combined content to: news.txt


Writing to file: 100%|██████████| 39996/39996 [00:00<00:00, 289915.72chunk/s]


✅ Successfully created news.txt
📁 Output location: /home/lang-chain/Documents/daraz_product_review/Notebook/news.txt
📊 Final file size: 72.56 MB


In [ ]:
import re
import unicodedata

def clean_nepali_text(text):
    """Clean Nepali text while preserving Devanagari punctuation."""
    
    # Step 1: Unicode normalization (NFC)
    text = unicodedata.normalize('NFC', text)
    
    # Step 2: Keep Nepali-specific characters
    allowed_pattern = re.compile(
        r'[^'
        r'a-zA-Z'                    # English
        r'\u0900-\u097F'             # Devanagari including ।॥
        r'0-9\u0966-\u096F'          # Digits
        r'\s'                        # Whitespace
        r'.,!?;:()\[\]{}\-\'\"/\\'   # Punctuation
        r']+'
    )
    text = allowed_pattern.sub(' ', text)
    
    # Step 3: Fix spacing
    text = re.sub(r' +', ' ', text)
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\t+', ' ', text)
    
    return text.strip()

# Process file in chunks to avoid memory crashes
print("📖 Processing news.txt in chunks...")

chunk_size = 1024 * 1024  # 1 MB chunks
buffer = []
total_original = 0
total_cleaned = 0

try:
    with open('news.txt', 'r', encoding='utf-8') as infile, \
         open('new_cleaned.txt', 'w', encoding='utf-8') as outfile:
        
        chunk_num = 0
        while True:
            chunk = infile.read(chunk_size)
            if not chunk:
                break
            
            chunk_num += 1
            total_original += len(chunk)
            
            # Clean chunk
            cleaned = clean_nepali_text(chunk)
            
            # Filter lines (minimum 5 chars)
            lines = [line.strip() for line in cleaned.split('\n') 
                     if len(line.strip()) >= 5]
            
            # Write to output
            if lines:
                output = '\n'.join(lines) + '\n'
                outfile.write(output)
                total_cleaned += len(output)
            
            # Progress indicator
            if chunk_num % 10 == 0:
                print(f"  Processed {chunk_num} chunks ({total_original:,} chars)...")
    
    print(f"\n✓ Done!")
    print(f"Original: {total_original:,} chars")
    print(f"Cleaned: {total_cleaned:,} chars")
    print(f"Removed: {total_original - total_cleaned:,} chars")
    
    # Show sample
    print("\n--- Sample of cleaned text ---")
    with open('news_cleaned.txt', 'r', encoding='utf-8') as f:
        print(f.read(500))

except FileNotFoundError:
    print("❌ Error: news.txt not found in current directory")
except MemoryError:
    print("❌ Still running out of memory. Try increasing chunk_size or processing smaller sections.")
except Exception as e:
    print(f"❌ Error: {e}")

📖 Processing news.txt in chunks...
  Processed 10 chunks (10,485,760 chars)...
  Processed 20 chunks (20,971,520 chars)...

✓ Done!
Original: 29,798,999 chars
Cleaned: 27,530,783 chars
Removed: 2,268,216 chars

--- Sample of cleaned text ---
❌ Error: news.txt not found in current directory


In [7]:
import hashlib
from tqdm import tqdm

file1 = 'wikipedia_ncc_corpus.txt'
file2 = 'new_cleaned.txt'
output_file = 'news_wikipedia_ncc_corpus.txt'

def line_hash(text):
    """Fast MD5 hash for dedupe."""
    return hashlib.md5(text.strip().encode('utf-8')).hexdigest()

seen = set()
total_written = 0
total_seen = 0

with open(output_file, 'w', encoding='utf-8') as out:
    for file in [file1, file2]:
        print(f"\nProcessing {file}...")

        with open(file, 'r', encoding='utf-8', errors='ignore') as f:
            for line in tqdm(f):
                line = line.strip()
                total_seen += 1

                if len(line) < 5:
                    continue

                h = line_hash(line)
                if h in seen:
                    continue

                seen.add(h)
                out.write(line + "\n")
                total_written += 1

print("\n✓ Combined + deduplicated successfully!")
print(f"Total lines seen: {total_seen:,}")
print(f"Unique lines written: {total_written:,}")
print(f"Output saved to: {output_file}")



Processing wikipedia_ncc_corpus.txt...


6610859it [00:29, 224942.28it/s]



Processing new_cleaned.txt...


97988it [00:00, 185291.71it/s]


✓ Combined + deduplicated successfully!
Total lines seen: 6,708,847
Unique lines written: 6,707,124
Output saved to: news_wikipedia_ncc_corpus.txt


In [ ]:

from datasets import load_dataset
import os
from tqdm import tqdm

def download_cc100_english(output_file='cc100_en.txt', max_lines=500000):
    """
    Download CC-100 English corpus
    """
    print(f"Downloading CC-100 English corpus (max {max_lines:,} lines)...")
    
    try:
        # CC-100 is available on Hugging Face
        dataset = load_dataset(
            'facebook/cc100',
            lang='en',
            streaming=True,  # Critical for large datasets
            trust_remote_code=True
        )
        
        lines_written = 0
        with open(output_file, 'w', encoding='utf-8') as f:
            for example in tqdm(dataset['train'], desc="Downloading"):
                text = example.get('text', '').strip()
                
                if text:
                    # Basic filtering
                    words = text.split()
                    if 3 <= len(words) <= 512:  # Reasonable length
                        f.write(text + '\n')
                        lines_written += 1
                        
                        if lines_written >= max_lines:
                            break
        
        # Get file size
        size_mb = os.path.getsize(output_file) / 1024 / 1024
        
        print(f"\n✓ Downloaded {lines_written:,} lines")
        print(f"✓ File size: {size_mb:.1f} MB")
        print(f"✓ Saved to: {output_file}")
        
        return output_file
        
    except Exception as e:
        print(f"Error: {e}")
        print("\nTrying alternative method...")
        return download_cc100_alternative(output_file, max_lines)

def download_cc100_alternative(output_file, max_lines):
    """Alternative CC-100 download method"""
    try:
        # Sometimes CC-100 is in different format
        dataset = load_dataset('allenai/c4', 'en', streaming=True)
        
        lines_written = 0
        with open(output_file, 'w', encoding='utf-8') as f:
            for example in tqdm(dataset['train'], desc="Downloading C4 (alternative)"):
                text = example.get('text', '').strip()
                if text:
                    f.write(text + '\n')
                    lines_written += 1
                    if lines_written >= max_lines:
                        break
        
        print(f"✓ Downloaded C4 English: {lines_written:,} lines")
        return output_file
        
    except Exception as e:
        print(f"Alternative failed: {e}")
        return None

# Download reasonable amount
download_cc100_english('cc100_en_200k.txt', max_lines=200000)

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'facebook/cc100' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


Error: Dataset 'facebook/cc100' doesn't exist on the Hub or cannot be accessed.

Trying alternative method...


Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

✓ Downloaded C4 English: 200,000 lines


'cc100_en_200k.txt'

In [3]:
import re
import unicodedata

def clean_english_text_simple(text):
    """Simple English text cleaner similar to your Nepali cleaner."""
    
    # Step 1: Unicode normalization
    text = unicodedata.normalize('NFKC', text)
    
    # Step 2: Keep only English characters, digits, and basic punctuation
    allowed_pattern = re.compile(
        r'[^'
        r'a-zA-Z'                    # English letters
        r'0-9'                       # Digits
        r'\s'                        # Whitespace
        r'.,!?;:()\-_\'"'            # Basic punctuation
        r']+'
    )
    text = allowed_pattern.sub(' ', text)
    
    # Step 3: Fix spacing
    text = re.sub(r' +', ' ', text)
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\t+', ' ', text)
    
    # Step 4: Remove lines that are too short or too long
    lines = text.split('\n')
    filtered_lines = []
    
    for line in lines:
        line = line.strip()
        if 20 <= len(line) <= 500:  # Reasonable line length
            # Check if has at least 3 words
            if len(line.split()) >= 3:
                filtered_lines.append(line)
    
    return '\n'.join(filtered_lines)

# Process file in chunks (identical to your Nepali processor)
def clean_english_file_simple(input_file, output_file, chunk_size=1024*1024):
    """Simple English file cleaner with chunk processing."""
    
    print(f"📖 Cleaning {input_file}...")
    
    total_original = 0
    total_cleaned = 0
    chunk_num = 0
    
    try:
        with open(input_file, 'r', encoding='utf-8', errors='ignore') as infile, \
             open(output_file, 'w', encoding='utf-8') as outfile:
            
            while True:
                chunk = infile.read(chunk_size)
                if not chunk:
                    break
                
                chunk_num += 1
                total_original += len(chunk)
                
                # Clean chunk
                cleaned = clean_english_text_simple(chunk)
                
                # Write if not empty
                if cleaned.strip():
                    outfile.write(cleaned + '\n')
                    total_cleaned += len(cleaned)
                
                # Progress
                if chunk_num % 10 == 0:
                    print(f"  Processed {chunk_num} chunks...")
    
    except FileNotFoundError:
        print(f"❌ Error: {input_file} not found")
        return
    except Exception as e:
        print(f"❌ Error: {e}")
        return
    
    print(f"\n✓ Done!")
    print(f"Original: {total_original:,} chars")
    print(f"Cleaned: {total_cleaned:,} chars")
    print(f"Removed: {total_original - total_cleaned:,} chars")
    
    # Show sample
    print("\n--- Sample ---")
    with open(output_file, 'r', encoding='utf-8') as f:
        sample = ''.join([f.readline() for _ in range(3)])
        print(sample[:500])

# Usage (just like your Nepali code)
if __name__ == "__main__":
    clean_english_file_simple(
        input_file='cc100_en_200k.txt',
        output_file='cc100_en_cleaned.txt',
        chunk_size=1024*1024  
    )

📖 Cleaning cc100_en_200k.txt...
  Processed 10 chunks...
  Processed 20 chunks...
  Processed 30 chunks...
  Processed 40 chunks...
  Processed 50 chunks...
  Processed 60 chunks...
  Processed 70 chunks...
  Processed 80 chunks...
  Processed 90 chunks...
  Processed 100 chunks...
  Processed 110 chunks...
  Processed 120 chunks...
  Processed 130 chunks...
  Processed 140 chunks...
  Processed 150 chunks...
  Processed 160 chunks...
  Processed 170 chunks...
  Processed 180 chunks...
  Processed 190 chunks...
  Processed 200 chunks...
  Processed 210 chunks...
  Processed 220 chunks...
  Processed 230 chunks...
  Processed 240 chunks...
  Processed 250 chunks...
  Processed 260 chunks...
  Processed 270 chunks...
  Processed 280 chunks...
  Processed 290 chunks...
  Processed 300 chunks...
  Processed 310 chunks...
  Processed 320 chunks...
  Processed 330 chunks...
  Processed 340 chunks...
  Processed 350 chunks...
  Processed 360 chunks...
  Processed 370 chunks...
  Processed 380

In [4]:
import hashlib
from tqdm import tqdm

file1 = 'news_wikipedia_ncc_corpus.txt'
file2 = 'cc100_en_cleaned.txt'
output_file = 'eng_news_wikipedia_ncc_corpus.txt'

def line_hash(text):
    """Fast MD5 hash for dedupe."""
    return hashlib.md5(text.strip().encode('utf-8')).hexdigest()

seen = set()
total_written = 0
total_seen = 0

with open(output_file, 'w', encoding='utf-8') as out:
    for file in [file1, file2]:
        print(f"\nProcessing {file}...")

        with open(file, 'r', encoding='utf-8', errors='ignore') as f:
            for line in tqdm(f):
                line = line.strip()
                total_seen += 1

                if len(line) < 5:
                    continue

                h = line_hash(line)
                if h in seen:
                    continue

                seen.add(h)
                out.write(line + "\n")
                total_written += 1

print("\n✓ Combined + deduplicated successfully!")
print(f"Total lines seen: {total_seen:,}")
print(f"Unique lines written: {total_written:,}")
print(f"Output saved to: {output_file}")



Processing news_wikipedia_ncc_corpus.txt...


6707124it [00:29, 225239.93it/s]



Processing cc100_en_cleaned.txt...


1515908it [00:02, 536323.05it/s]


✓ Combined + deduplicated successfully!
Total lines seen: 8,223,032
Unique lines written: 8,222,518
Output saved to: eng_news_wikipedia_ncc_corpus.txt


In [ ]:
import sentencepiece as spm
from tqdm import tqdm
import threading
import time
import os

print("Starting training...")

# Progress tracking using file size monitoring
class TrainingMonitor:
    def __init__(self, model_prefix):
        self.model_prefix = model_prefix
        self.model_file = f"{model_prefix}.model"
        self.running = True
        self.pbar = tqdm(total=100, desc="Training", unit="%", 
                         bar_format='{l_bar}{bar}| {elapsed}')
        
    def monitor(self):
        """Monitor training by checking if model file exists and grows"""
        last_size = 0
        start_time = time.time()
        
        while self.running:
            time.sleep(2)  # Check every 2 seconds
            
            if os.path.exists(self.model_file):
                current_size = os.path.getsize(self.model_file)
                if current_size > last_size:
                    # File is growing, training is progressing
                    last_size = current_size
                    elapsed = time.time() - start_time
                    self.pbar.set_postfix({"size": f"{current_size/1024:.1f}KB", 
                                          "time": f"{elapsed:.0f}s"})
        
    def start(self):
        self.thread = threading.Thread(target=self.monitor, daemon=True)
        self.thread.start()
        
    def stop(self):
        self.running = False
        self.pbar.n = 100
        self.pbar.refresh()
        self.pbar.close()

# Start monitoring
monitor = TrainingMonitor('ne_spm')
monitor.start()

try:
    # Train with optimized settings
    spm.SentencePieceTrainer.train(
    input='eng_news_wikipedia_ncc_corpus.txt',
    model_prefix='ne_spm_fixed',
    vocab_size=64000,
    character_coverage=0.9995,
    model_type='bpe',
    
    # ============ ADD THESE ============
    split_by_whitespace=True,        # Respect word boundaries
    split_by_unicode_script=True,    # Separate Devanagari/Latin scripts
    split_by_number=True,            # Keep numbers separate
    treat_whitespace_as_suffix=False, # Better word starts
    byte_fallback=True,              # Handle rare chars gracefully
    
    # Optionally: add common English words as user symbols
    user_defined_symbols=[
        'the', 'and', 'to', 'of', 'a', 'in', 'for', 'is', 'on', 'that',
        'book', 'flight', 'customer', 'service', 'help', 'want', 'need'
    ],
    # ===================================
    
    input_sentence_size=6707124,
    shuffle_input_sentence=True,
    max_sentence_length=8192,
    num_threads=16,
    minloglevel=1
)
finally:
    monitor.stop()

print("\n✓ Training complete!")
print(f"Model saved: ne_spm_fixed.model")
print(f"Vocab saved: ne_spm_fixed.vocab")

Starting training for eng_news_wikipedia_ncc_corpus (8.2M lines)...

📊 Analyzing corpus composition...
   File size: 3.64 GB
   English-only lines: 0.0%
   Nepali-only lines: 90.4%
   Mixed lines: 9.6%
   → Optimizing for bilingual tokenization...



Training 8.2M corpus:   0%|          | 0/100 [00:00<?, ]

In [2]:
import sentencepiece as spm
from tqdm import tqdm
import threading
import time
import os

print("Starting training...")

# Progress tracking using file size monitoring
class TrainingMonitor:
    def __init__(self, model_prefix):
        self.model_prefix = model_prefix
        self.model_file = f"{model_prefix}.model"
        self.running = True
        self.pbar = tqdm(total=100, desc="Training", unit="%", 
                         bar_format='{l_bar}{bar}| {elapsed}')
        
    def monitor(self):
        last_size = 0
        start_time = time.time()
        
        while self.running:
            time.sleep(2)
            if os.path.exists(self.model_file):
                current_size = os.path.getsize(self.model_file)
                if current_size > last_size:
                    last_size = current_size
                    elapsed = time.time() - start_time
                    self.pbar.set_postfix(
                        {"size": f"{current_size/1024:.1f}KB",
                         "time": f"{elapsed:.0f}s"}
                    )
        
    def start(self):
        self.thread = threading.Thread(target=self.monitor, daemon=True)
        self.thread.start()
        
    def stop(self):
        self.running = False
        self.pbar.n = 100
        self.pbar.refresh()
        self.pbar.close()

# Start monitoring (FIXED PREFIX)
monitor = TrainingMonitor('ne_spm_fixed')
monitor.start()

try:
    spm.SentencePieceTrainer.train(
        input='eng_news_wikipedia_ncc_corpus.txt',
        model_prefix='ne_spm_fixed',
        vocab_size=64000,
        character_coverage=0.9998,
        model_type='bpe',

        split_by_whitespace=True,
        split_by_unicode_script=True,
        split_by_number=True,
        treat_whitespace_as_suffix=False,
        byte_fallback=True,

        user_defined_symbols=[
            'the', 'and', 'to', 'of', 'a', 'in', 'for', 'is', 'on', 'that',
            'book', 'flight', 'customer', 'service', 'help', 'want', 'need'
        ],

        input_sentence_size=0,
        shuffle_input_sentence=True,
        max_sentence_length=8192,
        num_threads=4,    # Kaggle-safe
        minloglevel=0     # Show full logs
    )
finally:
    monitor.stop()

print("\n✓ Training complete!")
print("Model saved: ne_spm_fixed.model")
print("Vocab saved: ne_spm_fixed.vocab")


Starting training...


Training:   0%|          | 00:00

: 